In [1]:
import pandas as pd
import numpy as np
import random

In [2]:
df  = pd.read_csv("data/nyc_train_validate.csv")

In [28]:
df[df['education']==-9998]

,id,person_id,trip_n,travel_date,travel_date_dow,o_purpose_category,d_purpose_category,num_non_hh_travelers,num_hh_travelers,num_travelers,...,rent_own,income_aggregate,num_people,num_adults,num_kids,num_workers,num_students,disability,trip_distance,mode
386,386,16,0,03/06/2019,1,1,6,0,2,2,...,1,-9998,6,5,1,2,2,none,0.4,walk
387,387,16,1,03/06/2019,1,6,6,0,2,2,...,1,-9998,6,5,1,2,2,none,0.9,walk
388,388,16,2,03/06/2019,1,6,1,0,2,2,...,1,-9998,6,5,1,2,2,none,0.4,walk
389,389,16,3,05/06/2019,3,1,4,0,1,1,...,1,-9998,6,5,1,2,2,none,15.5,subway
390,390,16,4,05/06/2019,3,4,10,0,1,1,...,1,-9998,6,5,1,2,2,none,1.4,subway
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39292,39292,1786,16,12/06/2019,3,5,10,0,3,3,...,1,1,5,2,3,2,3,none,1.8,bus
39293,39293,1786,17,12/06/2019,3,1,7,0,2,2,...,1,1,5,2,3,2,3,none,0.5,drive
39294,39294,1786,18,12/06/2019,3,7,1,0,2,2,...,1,1,5,2,3,2,3,none,0.6,drive
39295,39295,1786,19,13/06/2019,4,1,5,0,4,4,...,1,1,5,2,3,2,3,none,2.3,drive


In [29]:
df.head()

,id,person_id,trip_n,travel_date,travel_date_dow,o_purpose_category,d_purpose_category,num_non_hh_travelers,num_hh_travelers,num_travelers,...,rent_own,income_aggregate,num_people,num_adults,num_kids,num_workers,num_students,disability,trip_distance,mode
0,0,0,0,30/05/2019,4,-9998,2,0,1,1,...,2,4,5,3,2,1,2,none,13.4,subway
1,1,0,1,01/06/2019,6,-9998,1,0,1,1,...,2,4,5,3,2,1,2,none,3.7,drive
2,2,0,2,02/06/2019,7,-9998,7,1,2,3,...,2,4,5,3,2,1,2,none,9.8,drive
3,3,0,3,02/06/2019,7,7,7,1,2,3,...,2,4,5,3,2,1,2,none,11.1,drive
4,4,0,4,03/06/2019,1,-9998,7,0,1,1,...,2,4,5,3,2,1,2,none,2.5,drive


In [14]:
df.iloc[:,:10].describe()

,id,person_id,trip_n,travel_date_dow,o_purpose_category,d_purpose_category,num_non_hh_travelers,num_hh_travelers,num_travelers
count,39444.000000,39444.000000,39444.000000,39444.000000,39444.000000,39444.000000,39444.000000,39444.000000,39444.00000
mean,19744.755172,876.586705,17.261662,4.019217,4.645624,5.368193,0.351334,1.313787,1.66512
std,11409.587711,519.076144,14.804657,1.969080,2.903368,3.145701,0.857035,0.685326,1.14115
min,0.000000,0.000000,0.000000,1.000000,1.000000,1.000000,0.000000,1.000000,1.00000
25%,9860.750000,431.000000,6.000000,2.000000,1.000000,2.000000,0.000000,1.000000,1.00000
50%,19721.500000,850.000000,14.000000,4.000000,6.000000,6.000000,0.000000,1.000000,1.00000
75%,29636.250000,1326.000000,25.000000,6.000000,7.000000,8.000000,0.000000,1.000000,2.00000
max,39497.000000,1794.000000,118.000000,7.000000,10.000000,10.000000,5.000000,7.000000,12.00000


In [15]:
df.iloc[:,10:21].describe()

,o_location_type,d_location_type,o_congestion,d_congestion,age,employment,student,license,planning_apps,industry,gender
count,39444.000000,39444.000000,39444.000000,39444.000000,39444.000000,39444.000000,39444.000000,39444.000000,39444.000000,39444.000000,39444.000000
mean,3.046801,3.186999,0.157337,0.124075,6.405638,2.383734,0.144483,0.850928,-428.989656,9.774719,1.487983
std,1.329513,1.276065,0.364123,0.329671,1.465089,2.070089,0.449391,0.356164,2034.571085,5.166532,0.527405
min,1.000000,1.000000,0.000000,0.000000,4.000000,1.000000,0.000000,0.000000,-9998.000000,1.000000,1.000000
25%,1.000000,2.000000,0.000000,0.000000,5.000000,1.000000,0.000000,1.000000,1.000000,7.000000,1.000000
50%,4.000000,4.000000,0.000000,0.000000,6.000000,1.000000,0.000000,1.000000,4.000000,9.000000,1.000000
75%,4.000000,4.000000,0.000000,0.000000,7.000000,3.000000,0.000000,1.000000,5.000000,14.000000,2.000000
max,4.000000,4.000000,1.000000,1.000000,11.000000,7.000000,2.000000,1.000000,8.000000,18.000000,4.000000


In [29]:
df.iloc[:,21:].describe()

,education,num_bicycles,num_vehicles,res_type,rent_own,income_aggregate,num_people,num_adults,num_kids,num_workers,num_students,trip_distance
count,39444.000000,39444.000000,39444.000000,39444.000000,39444.000000,39444.000000,39444.000000,39444.000000,39444.000000,39444.000000,39444.000000,39444.000000
mean,5.198661,0.864796,0.801110,4.070961,-1022.836604,3.572508,2.635635,2.097733,0.537902,1.545482,0.725763,2.574151
std,1.870387,1.211000,0.865651,1.835559,3032.300705,1.625121,1.437748,1.007089,0.907260,0.947141,1.020920,3.625045
min,1.000000,0.000000,0.000000,1.000000,-9998.000000,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.100000
25%,3.000000,0.000000,0.000000,3.000000,1.000000,2.000000,2.000000,1.000000,0.000000,1.000000,0.000000,0.300000
50%,6.000000,0.000000,1.000000,5.000000,2.000000,4.000000,2.000000,2.000000,0.000000,1.000000,0.000000,0.900000
75%,7.000000,1.000000,1.000000,6.000000,2.000000,5.000000,4.000000,2.000000,1.000000,2.000000,1.000000,3.300000
max,7.000000,8.000000,5.000000,9.000000,3.000000,6.000000,11.000000,8.000000,7.000000,6.000000,7.000000,20.000000


In [3]:
# delete rows without personid (54)
df = df[df['person_id']!=-9998]

## o_purpose_category

In [4]:
def process_o_purpose_category(df, row):
    '''
    impute by trips with the same person_id, travel date of week, d_purpose_category; otherwise, impute with all data by probability
    '''
    if row['o_purpose_category'] >= 0:
        return row['o_purpose_category']
    
    temp = df[(df['person_id']==row['person_id'])&\
              (df['travel_date_dow']==row['travel_date_dow'])&\
              (df['d_purpose_category']==row['d_purpose_category'])&\
              (df['o_purpose_category']!=-9998)]
    
    if temp.empty:
        o_purpose = df[(df['travel_date_dow']==row['travel_date_dow'])&\
                  (df['d_purpose_category']==row['d_purpose_category'])&\
                   (df['o_purpose_category']!=-9998)]['o_purpose_category']
        
        percentage = o_purpose.value_counts().cumsum() / o_purpose.value_counts().sum()
        random_number = random.uniform(0, 1)

        return percentage[percentage>random_number].index[0]
    
    return temp['o_purpose_category'].mode()[0]

In [5]:
random.seed(42)
df['o_purpose_category'] = df.apply(lambda x:process_o_purpose_category(df,x), axis=1)

In [174]:
df[df['o_purpose_category'].isna()]

,id,person_id,trip_n,travel_date,travel_date_dow,o_purpose_category,d_purpose_category,num_non_hh_travelers,num_hh_travelers,num_travelers,...,rent_own,income_aggregate,num_people,num_adults,num_kids,num_workers,num_students,disability,trip_distance,mode


## gender

In [6]:
gender = df[df['gender']!=-9998]['gender']
gender = gender.value_counts().cumsum() / gender.value_counts().sum()
def process_gender(df, row):
    if row['gender'] >= 0:
        return row['gender']
    random_number = random.uniform(0, 1)
    return gender[gender>random_number].index[0]

In [7]:
random.seed(42)
df['gender'] = df.apply(lambda x:process_gender(df,x), axis=1)

## license

In [8]:
def process_license(df, row):
    if row['license'] >= 0:
        return row['license']

    temp = df[(df['person_id']==row['person_id'])&\
              (df['license']!=-9998)]
    
    if temp.empty:
        license = df[(df['age']==row['age'])&\
                  (df['gender']==row['gender'])&\
                  (df['license']!=-9998)]['license']
        if license.empty:
            license = df[(df['age']==row['age'])&\
                      (df['license']!=-9998)]['license']
            
        percentage = license.value_counts().cumsum() / license.value_counts().sum()
        random_number = random.uniform(0, 1)
        return percentage[percentage>random_number].index[0]
    
    return temp['license'].mode()[0]    

In [9]:
random.seed(42)
df['license'] = df.apply(lambda x:process_license(df,x), axis=1)

In [157]:
df[df['license'].isna()]

,id,person_id,trip_n,travel_date,travel_date_dow,o_purpose_category,d_purpose_category,num_non_hh_travelers,num_hh_travelers,num_travelers,...,rent_own,income_aggregate,num_people,num_adults,num_kids,num_workers,num_students,disability,trip_distance,mode


## education

In [10]:
education = df[df['education']!=-9998]['education']
education = education.value_counts().cumsum() / education.value_counts().sum()
def process_education(df, row):
    if row['education'] >= 0:
        return row['education']
    random_number = random.uniform(0, 1)
    return education[education>=random_number].index[0]

In [11]:
random.seed(42)
df['education'] = df.apply(lambda x:process_education(df,x), axis=1)

## industry

In [92]:
#df['industry'] = df['industry'].replace([1,2,3,4,5,7,8,9,10,11,13,14,15,16,17], 18)

In [12]:
industry = df[df['industry']!=-9998]['industry']
industry = industry.value_counts().cumsum() / industry.value_counts().sum()
def process_industry(df, row):
    if row['industry'] >= 0:
        return row['industry']
    random_number = random.uniform(0, 1)
    return industry[industry>=random_number].index[0]

In [13]:
random.seed(42)
df['industry'] = df.apply(lambda x:process_industry(df,x), axis=1)

## income

In [19]:
def process_income(df, row):
    if row['income_aggregate'] >= 0:
        return row['income_aggregate']

    income = df[(df['age']==row['age'])&\
          (df['gender']==row['gender'])&\
          (df['education']==row['education'])&\
          (df['industry']==row['industry'])&\
          (df['income_aggregate']!=-9998)]['income_aggregate']

    
    if income.empty:
        income = df[(df['age']==row['age'])&\
              (df['education']==row['education'])&\
              (df['income_aggregate']!=-9998)]['income_aggregate']
                    
    percentage = income.value_counts().cumsum() / income.value_counts().sum()
    random_number = random.uniform(0, 1)
    return percentage[percentage>random_number].index[0] 

In [21]:
idid = df[df['income_aggregate']==-9998]['person_id'].unique()
df[(df['person_id'].isin(idid))&(df['income_aggregate']!=-9998)]

,id,person_id,trip_n,travel_date,travel_date_dow,o_purpose_category,d_purpose_category,num_non_hh_travelers,num_hh_travelers,num_travelers,...,rent_own,income_aggregate,num_people,num_adults,num_kids,num_workers,num_students,disability,trip_distance,mode


In [23]:
df['income_aggregate'] = df.apply(lambda x:process_income(df,x), axis=1)

## residence type

In [26]:
def process_restype(df, row):
    if row['res_type'] >= 0:
        return row['res_type']

    restype = df[(df['income_aggregate']==row['income_aggregate'])&\
              (df['num_people']==row['num_people'])&\
              (df['res_type']!=-9998)]['res_type']
    
    if restype.empty:
        restype = df[(df['income_aggregate']==row['income_aggregate'])&\
                  (df['res_type']!=-9998)]['res_type']
    
    
    percentage = restype.value_counts().cumsum() / restype.value_counts().sum()
    random_number = random.uniform(0, 1)
    return percentage[percentage>random_number].index[0]      

In [28]:
df['res_type'] = df.apply(lambda x:process_restype(df,x), axis=1)

In [30]:
df.to_csv("data/processed_data.csv",index=False)